# Doc2VecでWikipedia全文を使用

Wiki全文（か、もしくは一部）をDoc2Vecのインプットに指定して動作確認します。

## (1) Wikipediaコンテンツファイルから本文を抽出

### (1-1) Wikipediaコンテンツ取得

Wikipedia コンテンツの一覧を、以下のURLで参照します。

https://dumps.wikimedia.org/jawiki/latest/

このページのリストから、以下のファイルのリンクをクリックしてダウンロードします。

2.4GBほどになります。

```
jawiki-latest-pages-articles.xml.bz2    21-May-2017 01:07    2492605537
```

### (1-2) Wikipedia Extractor（本文抽出ツール）の取得

下記サイト内のDownloadsからWikipedia ExtractorのPythonスクリプトを直接ダウンロードします。

http://medialab.di.unipi.it/wiki/Wikipedia_Extractor

### (1-3) Wikipediaコンテンツファイルから本文を抽出

下記コマンドで抽出を実行します。

python3 WikiExtractor.py -b 10M -o extracted jawiki-latest-pages-articles.xml.bz2

これで、extractedディレクトリ下に、10MB毎に区切られた抽出後ファイルが生成されていきます。

extractedディレクトリ配下には、AA、AB、AC・・・といったサブディレクトリが生成されます。

サブディレクトリごとに、１００本のファイルが生成されます。

（すなわちサブディレクトリごとに1GBまで）

実行には４０分ほどかかるとのことでしたが、実績としては、5/22 13:51開始〜14:26終了、なので３５分ほどで完了しています。

実行例：
```
MacBookPro-makmorit-jp:Wikipedia makmorit$ pwd
/Users/makmorit/Documents/Development/Wikipedia
MacBookPro-makmorit-jp:Wikipedia makmorit$ ls -al
total 4868576
drwxr-xr-x   4 makmorit  staff         136 May 22 13:44 .
drwxr-xr-x  10 makmorit  staff         340 May 22 13:43 ..
-rw-r--r--@  1 makmorit  staff       98379 Mar 23  2016 WikiExtractor.py
-rw-r--r--@  1 makmorit  staff  2492605537 May 22 13:27 jawiki-latest-pages-articles.xml.bz2
MacBookPro-makmorit-jp:Wikipedia makmorit$ python WikiExtractor.py -b 10M -o extracted jawiki-latest-pages-articles.xml.bz2
INFO: Loaded 0 templates in 0.0s
INFO: Starting page extraction from jawiki-latest-pages-articles.xml.bz2.
INFO: Using 7 extract processes.
WARNING: Template errors in article '自然数' (4671): title(9) recursion(0, 0, 0)
WARNING: Template errors in article '階乗' (12763): title(1) recursion(0, 0, 0)
INFO: Extracted 10000 articles (225.3 art/s)
INFO: Extracted 20000 articles (263.3 art/s)
（中略）
WARNING: Template errors in article 'プロ経営者' (3516673): title(2) recursion(0, 0, 0)
INFO: Extracted 1410000 articles (990.3 art/s)
WARNING: Template errors in article '2017年のWTAツアー' (3535919): title(2) recursion(0, 0, 0)
INFO: Extracted 1420000 articles (995.9 art/s)
INFO: Extracted 1430000 articles (910.8 art/s)
WARNING: Template errors in article 'Wikipedia:削除依頼/性液' (3583405): title(1) recursion(0, 0, 0)
INFO: Extracted 1440000 articles (986.9 art/s)
INFO: Finished 7-process extraction of 1446777 articles in 2059.6s (702.5 art/s)
MacBookPro-makmorit-jp:Wikipedia makmorit$ date
Mon May 22 14:25:52 JST 2017
MacBookPro-makmorit-jp:Wikipedia makmorit$ 
```

## (2) 抽出コンテンツからTaggedDocumentを生成

### (2-1) 抽出コンテンツの内容についての注意

下記のように、タイトルに : 文字が含まれている場合は有効な記事でないようです。

```
<doc id="1" url="https://ja.wikipedia.org/wiki?curid=1" title="Wikipedia:アップロードログ 2004年4月">
Wikipedia:アップロードログ 2004年4月
<ul>
</doc>
```

したがってTaggedDocumentへの変換時は、これらのエントリーは除外する必要があります。

### (2-2) 抽出コンテンツをTaggedDocumentへ変換


有効なドキュメントは下記のようになっています。

```
<doc id="2466881" url="https://ja.wikipedia.org/wiki?curid=2466881" title="ヒントン駅">
ヒントン駅
ヒントン駅（ヒントンえき、英語：Hinton Station）は、ウェストバージニア州 メープル・アベニューのセカンド・アヴェニュー100にある駅。 昔のチェサピーク・アンド・オハイオ鉄道の駅である。
アムトラックの停車する列車は下記の通り。
</doc>
```

これを、以下のようにして、TaggedDocumentに変換します。

分ち書きにはプロダクションのNlangクラスを使用します（すなわち品詞を一部落とします）。

In [1]:
'''
    環境準備
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

'''
    コンテンツファイルから逐一読み込み、
    タグとコーパスをTaggedDocumentに設定
'''
from gensim.models.doc2vec import TaggedDocument
from learning.core.nlang import Nlang


# これは <doc> タグのidです
tag = 2466881

# これは <doc> タグの中身です
sentences = '''ヒントン駅
ヒントン駅（ヒントンえき、英語：Hinton Station）は、ウェストバージニア州 メープル・アベニューのセカンド・アヴェニュー100にある駅。 
昔のチェサピーク・アンド・オハイオ鉄道の駅である。
アムトラックの停車する列車は下記の通り。
'''

separated_sentences = Nlang.split(sentences)
words = separated_sentences.split()

tagged_document = TaggedDocument(words=words, tags=[tag])
tagged_document

TaggedDocument(words=['ヒントン', '駅', 'ヒントン', '駅', 'ヒントン', 'える', '英語', 'Ｈｉｎｔｏｎ', 'Ｓｔａｔｉｏｎ', 'ウェストバージニア', '州', 'メープル・アベニュー', 'セカンド・アヴェニュー', '１００', '駅', '昔', 'チェサピーク・アンド・オハイオ', '鉄道', '駅', 'アムトラック', '停車', 'する', '列車', '下記', '通り'], tags=[2466881])

#### 全量の変換は未実施（上記処理を、抽出したWikipediaの全コンテンツについて実施する予定）。詳細は後報いたします。

### (2-3) TaggedDocumentをインプットとして学習

今回はマイオペのテストデータは入れないでテストします。

概ね下記のようなコードを実行して学習します。

ただし分類数がわからないので、適当に500程度のサイズに設定しておきます。

繰り返し回数はサイズの１０倍程度を指定します。

In [ ]:
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

def train(tagged_document_list, dm=0):
    '''
        tagged_document_listは、
        すべてのWikipediaのタグ付きコーパス
        (前述のtagged_document)が収容されたリスト。

        これを引数にして、
        ボキャブラリ生成／学習実行を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
        （今回は比較のため、両方で試す予定）
    '''
    model = Doc2Vec(dm=dm, size=500, min_count=1, iter=5000)
    model.build_vocab(tagged_document_list)
    ret = model.train(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('train_by_doc2vec: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [ ]:
'''
    tagged_document_listを
    使用し、学習実行
'''
train(tagged_document_list) 

#### 結果は後報します。

### (2-4) 適当な質問文で予測実行

概ね下記のようなコードを実行して学習します。


In [ ]:
def predict(word, dm=0):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(dm))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [ ]:
'''
    契約書を見たいのですが
'''
predict('契約書を見たいのですが')

#### 結果は後報します。